In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_bitsp_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,How To Get Voice Over Work Through @voyzappin ...,Harsh Joshi,1137944535722471424,2019-06-10 04:48:13,"['voiceover', 'voiceoverartist', 'voiceoverwork']",HJ_Voice,0,0,0.0,0.7184,Positive,5
1,1,"@RadhaManisha Hi, I'm from Bits-Pilani. I came...",anirudh,1137824159289884672,2019-06-09 20:49:53,[],nsanirudh,0,0,0.0,0.7598,Positive,5
2,2,HDC meets many years later :):) and it's a rio...,Brajesh Bajpai,1137109070006104064,2019-06-07 21:28:23,"['rakeshnamgia', 'bitspilani']",Brajesh_Bajpai,0,0,0.0,0.7184,Positive,5
3,3,"Respected sir, Kindly do the needful to valida...",vinod,1136890432674246656,2019-06-07 06:59:36,[],vinod32822142,0,0,0.0,0.8316,Positive,5
4,4,There are so many working professionals who ha...,sudipta subudhi,1136884902912151552,2019-06-07 06:37:37,[],sudipta_subudhi,2,2,0.0,0.0000,Neutral,3
5,5,"True, and BITS Pilani Campus as well - but tho...",www.TheLearningPoint.net 😺🎃🐋🐊🐙,1136875802002198528,2019-06-07 06:01:27,[],learning_pt,0,1,0.0,0.3506,Positive,4
6,6,"Respected Sir, Kindly see my request w.r.t. Mt...",vinod,1136875048512479232,2019-06-07 05:58:28,[],vinod32822142,0,0,0.0,0.8885,Positive,5
7,7,do you guyss happen to know someone in bits pi...,ad;tya l messikaandhhbhakt,1136690861155241984,2019-06-06 17:46:34,[],adityamaera,0,0,0.0,0.0000,Neutral,3
8,8,Rouse Avenue Court \nCMM Navin Kumar Kashyap h...,India Legal,1136571381016014848,2019-06-06 09:51:48,[],indialegalmedia,1,1,0.0,-0.7783,Negative,1
9,9,"14 years ago, this officer from Tamil Nadu, al...",Dennis S. Jesudasan,1136557942713413632,2019-06-06 08:58:24,[],DennisJesudasan,31,102,0.0,0.0000,Neutral,3


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,How To Get Voice Over Work Through @voyzappin ...,Harsh Joshi,1137944535722471424,2019-06-10 04:48:13,"['voiceover', 'voiceoverartist', 'voiceoverwork']",HJ_Voice,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000
1,"@RadhaManisha Hi, I'm from Bits-Pilani. I came...",anirudh,1137824159289884672,2019-06-09 20:49:53,[],nsanirudh,0.0,0.0,0.0,0.7598,Positive,5,0.000000,0.000000
2,HDC meets many years later :):) and it's a rio...,Brajesh Bajpai,1137109070006104064,2019-06-07 21:28:23,"['rakeshnamgia', 'bitspilani']",Brajesh_Bajpai,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000
3,"Respected sir, Kindly do the needful to valida...",vinod,1136890432674246656,2019-06-07 06:59:36,[],vinod32822142,0.0,0.0,0.0,0.8316,Positive,5,0.000000,0.000000
4,There are so many working professionals who ha...,sudipta subudhi,1136884902912151552,2019-06-07 06:37:37,[],sudipta_subudhi,2.0,2.0,0.0,0.0000,Neutral,3,0.006803,0.001742
5,"True, and BITS Pilani Campus as well - but tho...",www.TheLearningPoint.net 😺🎃🐋🐊🐙,1136875802002198528,2019-06-07 06:01:27,[],learning_pt,0.0,1.0,0.0,0.3506,Positive,4,0.000000,0.000871
6,"Respected Sir, Kindly see my request w.r.t. Mt...",vinod,1136875048512479232,2019-06-07 05:58:28,[],vinod32822142,0.0,0.0,0.0,0.8885,Positive,5,0.000000,0.000000
7,do you guyss happen to know someone in bits pi...,ad;tya l messikaandhhbhakt,1136690861155241984,2019-06-06 17:46:34,[],adityamaera,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000
8,Rouse Avenue Court \nCMM Navin Kumar Kashyap h...,India Legal,1136571381016014848,2019-06-06 09:51:48,[],indialegalmedia,1.0,1.0,0.0,-0.7783,Negative,1,0.003401,0.000871
9,"14 years ago, this officer from Tamil Nadu, al...",Dennis S. Jesudasan,1136557942713413632,2019-06-06 08:58:24,[],DennisJesudasan,31.0,102.0,0.0,0.0000,Neutral,3,0.105442,0.088850


In [6]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [7]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [8]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [9]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [10]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,how to get voice over work through @voyzappin ...,Harsh Joshi,1137944535722471424,2019-06-10 04:48:13,"['voiceover', 'voiceoverartist', 'voiceoverwork']",HJ_Voice,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000,0.000000,How To Get Voice Over Work Through @voyzappin ...,"[get, voice, work, voyzappin, voyzapp, com, st..."
1,"@radhamanisha hi, i'm from bits-pilani. i came...",anirudh,1137824159289884672,2019-06-09 20:49:53,[],nsanirudh,0.0,0.0,0.0,0.7598,Positive,5,0.000000,0.000000,0.000000,@RadhaManisha Hi I'm from Bits-Pilani I came a...,"[radhamanisha, hi, bits, pilani, came, across,..."
2,hdc meets many years later :):) and it's a rio...,Brajesh Bajpai,1137109070006104064,2019-06-07 21:28:23,"['rakeshnamgia', 'bitspilani']",Brajesh_Bajpai,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000,0.000000,HDC meets many years later :): and it's a riot...,"[hdc, meets, many, years, later, riot, many, s..."
3,"respected sir, kindly do the needful to valida...",vinod,1136890432674246656,2019-06-07 06:59:36,[],vinod32822142,0.0,0.0,0.0,0.8316,Positive,5,0.000000,0.000000,0.000000,Respected sir Kindly do the needful to validat...,"[respected, sir, kindly, needful, validate, mt..."
4,there are so many working professionals who ha...,sudipta subudhi,1136884902912151552,2019-06-07 06:37:37,[],sudipta_subudhi,2.0,2.0,0.0,0.0000,Neutral,3,0.006803,0.001742,0.477850,There are so many working professionals who ha...,"[many, working, professionals, currently, purs..."
5,"true, and bits pilani campus as well - but tho...",www.TheLearningPoint.net 😺🎃🐋🐊🐙,1136875802002198528,2019-06-07 06:01:27,[],learning_pt,0.0,1.0,0.0,0.3506,Positive,4,0.000000,0.000871,0.034843,True and BITS Pilani Campus as well but those ...,"[true, bits, pilani, campus, well, relatively,..."
6,"respected sir, kindly see my request w.r.t. mt...",vinod,1136875048512479232,2019-06-07 05:58:28,[],vinod32822142,0.0,0.0,0.0,0.8885,Positive,5,0.000000,0.000000,0.000000,Respected Sir Kindly see my request w.r.t Mtec...,"[respected, sir, kindly, see, request, w, r, m..."
7,do you guyss happen to know someone in bits pi...,ad;tya l messikaandhhbhakt,1136690861155241984,2019-06-06 17:46:34,[],adityamaera,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,do you guyss happen to know someone in bits pi...,"[guyss, happen, know, someone, bits, pilani]"
8,rouse avenue court \ncmm navin kumar kashyap h...,India Legal,1136571381016014848,2019-06-06 09:51:48,[],indialegalmedia,1.0,1.0,0.0,-0.7783,Negative,1,0.003401,0.000871,0.238925,Rouse Avenue Court CMM Navin Kumar Kashyap has...,"[rouse, avenue, court, cmm, navin, kumar, kash..."
9,"14 years ago, this officer from tamil nadu, al...",Dennis S. Jesudasan,1136557942713413632,2019-06-06 08:58:24,[],DennisJesudasan,31.0,102.0,0.0,0.0000,Neutral,3,0.105442,0.088850,9.880538,14 years ago this officer from Tamil Nadu also...,"[14, years, ago, officer, tamil, nadu, also, a..."


In [11]:
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,how to get voice over work through @voyzappin ...,Harsh Joshi,1137944535722471424,2019-06-10 04:48:13,"['voiceover', 'voiceoverartist', 'voiceoverwork']",HJ_Voice,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000,0.000000,How To Get Voice Over Work Through @voyzappin ...,"[get, voice, work, voyzappin, voyzapp, com, st...","[(get, VB), (voice, NN), (work, NN), (voyzappi..."
1,"@radhamanisha hi, i'm from bits-pilani. i came...",anirudh,1137824159289884672,2019-06-09 20:49:53,[],nsanirudh,0.0,0.0,0.0,0.7598,Positive,5,0.000000,0.000000,0.000000,@RadhaManisha Hi I'm from Bits-Pilani I came a...,"[radhamanisha, hi, bits, pilani, came, across,...","[(radhamanisha, NN), (hi, NN), (bits, NNS), (p..."
2,hdc meets many years later :):) and it's a rio...,Brajesh Bajpai,1137109070006104064,2019-06-07 21:28:23,"['rakeshnamgia', 'bitspilani']",Brajesh_Bajpai,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000,0.000000,HDC meets many years later :): and it's a riot...,"[hdc, meets, many, years, later, riot, many, s...","[(hdc, JJ), (meets, NNS), (many, JJ), (years, ..."
3,"respected sir, kindly do the needful to valida...",vinod,1136890432674246656,2019-06-07 06:59:36,[],vinod32822142,0.0,0.0,0.0,0.8316,Positive,5,0.000000,0.000000,0.000000,Respected sir Kindly do the needful to validat...,"[respected, sir, kindly, needful, validate, mt...","[(respected, VBN), (sir, NNS), (kindly, RB), (..."
4,there are so many working professionals who ha...,sudipta subudhi,1136884902912151552,2019-06-07 06:37:37,[],sudipta_subudhi,2.0,2.0,0.0,0.0000,Neutral,3,0.006803,0.001742,0.477850,There are so many working professionals who ha...,"[many, working, professionals, currently, purs...","[(many, JJ), (working, VBG), (professionals, N..."
5,"true, and bits pilani campus as well - but tho...",www.TheLearningPoint.net 😺🎃🐋🐊🐙,1136875802002198528,2019-06-07 06:01:27,[],learning_pt,0.0,1.0,0.0,0.3506,Positive,4,0.000000,0.000871,0.034843,True and BITS Pilani Campus as well but those ...,"[true, bits, pilani, campus, well, relatively,...","[(true, JJ), (bits, NNS), (pilani, VBP), (camp..."
6,"respected sir, kindly see my request w.r.t. mt...",vinod,1136875048512479232,2019-06-07 05:58:28,[],vinod32822142,0.0,0.0,0.0,0.8885,Positive,5,0.000000,0.000000,0.000000,Respected Sir Kindly see my request w.r.t Mtec...,"[respected, sir, kindly, see, request, w, r, m...","[(respected, VBN), (sir, NNS), (kindly, RB), (..."
7,do you guyss happen to know someone in bits pi...,ad;tya l messikaandhhbhakt,1136690861155241984,2019-06-06 17:46:34,[],adityamaera,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,do you guyss happen to know someone in bits pi...,"[guyss, happen, know, someone, bits, pilani]","[(guyss, NN), (happen, VB), (know, VBP), (some..."
8,rouse avenue court \ncmm navin kumar kashyap h...,India Legal,1136571381016014848,2019-06-06 09:51:48,[],indialegalmedia,1.0,1.0,0.0,-0.7783,Negative,1,0.003401,0.000871,0.238925,Rouse Avenue Court CMM Navin Kumar Kashyap has...,"[rouse, avenue, court, cmm, navin, kumar, kash...","[(rouse, NN), (avenue, NN), (court, NN), (cmm,..."
9,"14 years ago, this officer from tamil nadu, al...",Dennis S. Jesudasan,1136557942713413632,2019-06-06 08:58:24,[],DennisJesudasan,31.0,102.0,0.0,0.0000,Neutral,3,0.105442,0.088850,9.880538,14 years ago this officer from Tamil Nadu also...,"[14, years, ago, officer, tamil, nadu, also, a...","[(14, CD), (years, NNS), (ago, RB), (officer, ..."


In [12]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,how to get voice over work through @voyzappin ...,Harsh Joshi,1137944535722471424,2019-06-10 04:48:13,"['voiceover', 'voiceoverartist', 'voiceoverwork']",HJ_Voice,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000,0.000000,How To Get Voice Over Work Through @voyzappin ...,"[get, voice, work, voyzappin, voyzapp, com, st...","[(get, VB), (voice, NN), (work, NN), (voyzappi...","[voice, work, voyzappin, voyzapp, com, startup..."
1,"@radhamanisha hi, i'm from bits-pilani. i came...",anirudh,1137824159289884672,2019-06-09 20:49:53,[],nsanirudh,0.0,0.0,0.0,0.7598,Positive,5,0.000000,0.000000,0.000000,@RadhaManisha Hi I'm from Bits-Pilani I came a...,"[radhamanisha, hi, bits, pilani, came, across,...","[(radhamanisha, NN), (hi, NN), (bits, NNS), (p...","[radhamanisha, hi, bits, name, official, music..."
2,hdc meets many years later :):) and it's a rio...,Brajesh Bajpai,1137109070006104064,2019-06-07 21:28:23,"['rakeshnamgia', 'bitspilani']",Brajesh_Bajpai,0.0,0.0,0.0,0.7184,Positive,5,0.000000,0.000000,0.000000,HDC meets many years later :): and it's a riot...,"[hdc, meets, many, years, later, riot, many, s...","[(hdc, JJ), (meets, NNS), (many, JJ), (years, ...","[meets, years, stories, fun, loads, friends, r..."
3,"respected sir, kindly do the needful to valida...",vinod,1136890432674246656,2019-06-07 06:59:36,[],vinod32822142,0.0,0.0,0.0,0.8316,Positive,5,0.000000,0.000000,0.000000,Respected sir Kindly do the needful to validat...,"[respected, sir, kindly, needful, validate, mt...","[(respected, VBN), (sir, NNS), (kindly, RB), (...","[sir, validate, mtech, degree, bits, wilp, mode]"
4,there are so many working professionals who ha...,sudipta subudhi,1136884902912151552,2019-06-07 06:37:37,[],sudipta_subudhi,2.0,2.0,0.0,0.0000,Neutral,3,0.006803,0.001742,0.477850,There are so many working professionals who ha...,"[many, working, professionals, currently, purs...","[(many, JJ), (working, VBG), (professionals, N...","[professionals, tech, bits, course, maintains,..."
5,"true, and bits pilani campus as well - but tho...",www.TheLearningPoint.net 😺🎃🐋🐊🐙,1136875802002198528,2019-06-07 06:01:27,[],learning_pt,0.0,1.0,0.0,0.3506,Positive,4,0.000000,0.000871,0.034843,True and BITS Pilani Campus as well but those ...,"[true, bits, pilani, campus, well, relatively,...","[(true, JJ), (bits, NNS), (pilani, VBP), (camp...","[bits, campus, things]"
6,"respected sir, kindly see my request w.r.t. mt...",vinod,1136875048512479232,2019-06-07 05:58:28,[],vinod32822142,0.0,0.0,0.0,0.8885,Positive,5,0.000000,0.000000,0.000000,Respected Sir Kindly see my request w.r.t Mtec...,"[respected, sir, kindly, see, request, w, r, m...","[(respected, VBN), (sir, NNS), (kindly, RB), (...","[sir, r, mtech, validation, issue, wilp, mode,..."
7,do you guyss happen to know someone in bits pi...,ad;tya l messikaandhhbhakt,1136690861155241984,2019-06-06 17:46:34,[],adityamaera,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,do you guyss happen to know someone in bits pi...,"[guyss, happen, know, someone, bits, pilani]","[(guyss, NN), (happen, VB), (know, VBP), (some...","[guyss, someone, bits]"
8,rouse avenue court \ncmm navin kumar kashyap h...,India Legal,1136571381016014848,2019-06-06 09:51:48,[],indialegalmedia,1.0,1.0,0.0,-0.7783,Negative,1,0.003401,0.000871,0.238925,Rouse Avenue Court CMM Navin Kumar Kashyap has...,"[rouse, avenue, court, cmm, navin, kumar, kash...","[(rouse, NN), (avenue, NN), (court, NN), (cmm,...","[rouse, avenue, court, cmm, kumar, kashyap, cu..."
9,"14 years ago, this officer from tamil nadu, al...",Dennis S. Jesudasan,1136557942713413632,2019-06-06 08:58:24,[],DennisJesudasan,31.0,102.0,0.0,0.0000,Neutral,3,0.105442,0.088850,9.880538,14 years ago this officer from Tamil Nadu also...,"[14, years, ago, officer, tamil, nadu, also, a...","[(14, CD), (years, NNS), (ag

In [13]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,voice
1,work
2,voyzappin
3,voyzapp
4,com
5,startup
6,bits
7,amazon
8,marketplace
9,voiceover


In [14]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,voice,voice,5
1,work,work,69
2,voyzappin,voyzappin,2
3,voyzapp,voyzapp,2
4,com,com,1077
5,startup,startup,59
6,bits,bits,2523
7,amazon,amazon,4
8,marketplace,marketplace,8
9,voiceover,voiceover,2


In [15]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
6,bits,bits,2523
4,com,com,1077
131,twitter,twitter,1070
130,pic,pic,451
34,bitspilani,bitspilani,371
52,campus,campus,338
162,students,students,219
82,pilani,pilani,206
259,research,research,196
342,university,university,179


In [16]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [17]:
l=['youtube','hcl','washington','rajasthanpalaces','rajasthantourism','rajasthaniculture']

In [18]:
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))

In [19]:
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
45,economic condition and climate of india: dr. s...,Virat Hindustan Sangam,1134684755298009088,2019-06-01 04:55:01,[],vhsindia,23.0,66.0,0.0,0.0000,Neutral,3,0.078231,0.057491,6.993529,Economic condition and climate of India Dr Sub...,"[economic, condition, climate, india, dr, subr...","[(economic, JJ), (condition, NN), (climate, NN...","[condition, climate, india, dr, conversation, ...",1
172,birla institute of technology and science (bit...,Rajasthan Travel and Tourism Guide,1180574318691962880,2019-10-05 20:03:46,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Birla Institute of Technology and science (Bit...,"[birla, institute, technology, science, bits, ...","[(birla, NNS), (institute, VBP), (technology, ...","[birla, technology, science, bits, rajasthanpa...",3
192,birla institute of technology and science (bit...,Rajasthan Travel and Tourism Guide,1180574318691962880,2019-10-05 20:03:46,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Birla Institute of Technology and science (Bit...,"[birla, institute, technology, science, bits, ...","[(birla, NNS), (institute, VBP), (technology, ...","[birla, technology, science, bits, rajasthanpa...",3
281,birla institute of technology and science (bit...,Rajasthan Travel and Tourism Guide,1200384708237901824,2019-11-29 12:03:11,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,1.0,0.0,0.0000,Neutral,3,0.000000,0.000871,0.034843,Birla Institute of Technology and science (Bit...,"[birla, institute, technology, science, bits, ...","[(birla, NNS), (institute, VBP), (technology, ...","[birla, technology, science, bits, rajasthanpa...",3
328,birla institute of technology and science (bit...,Rajasthan Travel and Tourism Guide,1208719690299662336,2019-12-22 12:03:25,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,2.0,0.0,0.0000,Neutral,3,0.000000,0.001742,0.069686,Birla Institute of Technology and science (Bit...,"[birla, institute, technology, science, bits, ...","[(birla, NNS), (institute, VBP), (technology, ...","[birla, technology, science, bits, rajasthanpa...",3
403,birla institute of technology and science (bit...,Rajasthan Travel and Tourism Guide,1162454807967195136,2019-08-16 20:03:18,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,1.0,2.0,0.0,0.0000,Neutral,3,0.003401,0.001742,0.273768,Birla Institute of Technology and science (Bit...,"[birla, institute, technology, science, bits, ...","[(birla, NNS), (institute, VBP), (technology, ...","[birla, technology, science, bits, rajasthanpa...",3
426,have just returned from visiting bits-pilani a...,Anupal Bharali,1206864822219890688,2019-12-17 09:12:50,[],BharaliAnupal,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Have just returned from visiting BITS-PILANI a...,"[returned, visiting, bits, pilani, coral, jubi...","[(returned, VBN), (visiting, VBG), (bits, NNS)...","[bits, jubilee, reunion, batch, youtube]",1
444,have just returned from visiting bits-pilani a...,Anupal Bharali,1206864822219890688,2019-12-17 09:12:50,[],BharaliAnupal,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Have just returned from visiting BITS-PILANI a...,"[returned, visiting, bits, pilani, coral, jubi...","[(returned, VBN), (visiting, VBG), (bits, NNS)...","[bits, jubilee, reunion, batch, youtube]",1
811,birla institute of technology and science (bit...,Rajasthan Travel and Tourism Guide,1142040711249850368,2019-06-21 12:04:58,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,2.0,3.0,0.0,0.0000,Neutral,3,0.006803,0.002613,0.512693,Birla Institute of Technology and science (Bit...,"[birla, institute, technology, science, bits, ...","[(birla, NNS), (i

In [20]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [21]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [22]:
df['new_score']=df['score']
df_eng1=df[df.engagement==1]
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df[df.engagement==1]=df_eng1

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [24]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [25]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,how to get voice over work through @voyzappin ...,Harsh Joshi,1137944535722471424,2019-06-10 04:48:13,"['voiceover', 'voiceoverartist', 'voiceoverwork']",HJ_Voice,0.0,0.0,0.0,0.7184,...,0.000000,0.000000,0.000000,How To Get Voice Over Work Through @voyzappin ...,"[get, voice, work, voyzappin, voyzapp, com, st...","[(get, VB), (voice, NN), (work, NN), (voyzappi...","[voice, work, voyzappin, voyzapp, com, startup...",0,0,5
1,"@radhamanisha hi, i'm from bits-pilani. i came...",anirudh,1137824159289884672,2019-06-09 20:49:53,[],nsanirudh,0.0,0.0,0.0,0.7598,...,0.000000,0.000000,0.000000,@RadhaManisha Hi I'm from Bits-Pilani I came a...,"[radhamanisha, hi, bits, pilani, came, across,...","[(radhamanisha, NN), (hi, NN), (bits, NNS), (p...","[radhamanisha, hi, bits, name, official, music...",0,0,5
2,hdc meets many years later :):) and it's a rio...,Brajesh Bajpai,1137109070006104064,2019-06-07 21:28:23,"['rakeshnamgia', 'bitspilani']",Brajesh_Bajpai,0.0,0.0,0.0,0.7184,...,0.000000,0.000000,0.000000,HDC meets many years later :): and it's a riot...,"[hdc, meets, many, years, later, riot, many, s...","[(hdc, JJ), (meets, NNS), (many, JJ), (years, ...","[meets, years, stories, fun, loads, friends, r...",0,0,5
3,"respected sir, kindly do the needful to valida...",vinod,1136890432674246656,2019-06-07 06:59:36,[],vinod32822142,0.0,0.0,0.0,0.8316,...,0.000000,0.000000,0.000000,Respected sir Kindly do the needful to validat...,"[respected, sir, kindly, needful, validate, mt...","[(respected, VBN), (sir, NNS), (kindly, RB), (...","[sir, validate, mtech, degree, bits, wilp, mode]",0,0,5
4,there are so many working professionals who ha...,sudipta subudhi,1136884902912151552,2019-06-07 06:37:37,[],sudipta_subudhi,2.0,2.0,0.0,0.0000,...,0.006803,0.001742,0.477850,There are so many working professionals who ha...,"[many, working, professionals, currently, purs...","[(many, JJ), (working, VBG), (professionals, N...","[professionals, tech, bits, course, maintains,...",0,1,3
5,"true, and bits pilani campus as well - but tho...",www.TheLearningPoint.net 😺🎃🐋🐊🐙,1136875802002198528,2019-06-07 06:01:27,[],learning_pt,0.0,1.0,0.0,0.3506,...,0.000000,0.000871,0.034843,True and BITS Pilani Campus as well but those ...,"[true, bits, pilani, campus, well, relatively,...","[(true, JJ), (bits, NNS), (pilani, VBP), (camp...","[bits, campus, things]",0,0,4
6,"respected sir, kindly see my request w.r.t. mt...",vinod,1136875048512479232,2019-06-07 05:58:28,[],vinod32822142,0.0,0.0,0.0,0.8885,...,0.000000,0.000000,0.000000,Respected Sir Kindly see my request w.r.t Mtec...,"[respected, sir, kindly, see, request, w, r, m...","[(respected, VBN), (sir, NNS), (kindly, RB), (...","[sir, r, mtech, validation, issue, wilp, mode,...",0,0,5
7,do you guyss happen to know someone in bits pi...,ad;tya l messikaandhhbhakt,1136690861155241984,2019-06-06 17:46:34,[],adityamaera,0.0,0.0,0.0,0.0000,...,0.000000,0.000000,0.000000,do you guyss happen to know someone in bits pi...,"[guyss, happen, know, someone, bits, pilani]","[(guyss, NN), (happen, VB), (know, VBP), (some...","[guyss, someone, bits]",0,0,3
8,rouse avenue court \ncmm navin kumar kashyap h...,India Legal,1136571381016014848,2019-06-06 09:51:48,[],indialegalmedia,1.0,1.0,0.0,-0.7783,...,0.003401,0.000871,0.238925,Rouse Avenue Court CMM Navin Kumar Kashyap has...,"[rouse, avenue, court, cmm, navin, kumar, kash...","[(rouse, NN), (avenue, NN), (court, NN), (cmm,...","[rouse, avenue, court, cmm, kumar, kashyap, cu...",0,1,1
9,"14 years ago, this officer from tamil nadu, al...",Dennis S. Jesudasan,1136557942713413632,2019-06-06 08:58:24,[],DennisJesudasan,31.0,102.0,0.0,0.0000,...,0.105442,0.088850,9.880538,14 years ago this officer from Tamil Nadu also...,"[14, years, ago, officer, tamil, nadu, also, a...","[(14, CD), (ye

In [26]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/BITSPWeightedSentiment.csv')